In [1]:
import cv2
import os
import numpy as np
import imutils

In [2]:
def get_angles(amount,angle = 90):
    results = []
    for x in range(amount):
        results.append(x * angle)
    
    # return [x * angle for x in range(amount)]
    return results

In [3]:
def clahe(image):
    bgr = image
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=5)
    as_list = list(lab_planes)
    as_list[0] = clahe.apply(lab_planes[0])

    new_lab = tuple(as_list)
    lab = cv2.merge(new_lab)
    final_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    return final_image

In [4]:
import os
import cv2
import numpy as np

def make_square(image, desired_size):
    """
    Make an image square by padding it to the desired size without cropping.

    Parameters:
        image (numpy.ndarray): Input image.
        desired_size (int): Desired size of the square image.
        fill_color (tuple): Color for the padding, defaults to white (255, 255, 255).

    Returns:
        numpy.ndarray: Square image.
    """
    old_size = image.shape[:2]
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # Resize the image while preserving aspect ratio
    image = cv2.resize(image, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    # Define the border size
   

    # Extract corner pixels' colors
    top_left_color = image[0, 0]
    top_right_color = image[0, -1]
    bottom_left_color = image[-1, 0]
    bottom_right_color = image[-1, -1]

    # Use the corner color as the border color
    border_color = np.mean([top_left_color, top_right_color, bottom_left_color, bottom_right_color], axis=0).astype(int)


    # Create a border around the resized image with white color
    new_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=border_color.tolist())
    return new_image


In [5]:
def work_augmentation(input_array):
    # angles = get_angles(5)
    angles = get_angles(3)
    for folder in input_array:
        step = 1
        print("[Info] Access to %s ..." % (folder["name"]))
        asset_path = folder['input']
        output_path = os.path.join("output","mango","augmentation-enhancement",folder["name"])
        
        # check if output folder is existing, so we can create new one!
        print("[Step %d] - Checking output folder..." % (step))
        if not os.path.isdir(output_path):
            print("[Error Step %d] - Output path is not exist, Please running code after folder creation are done." % (step))
            os.makedirs(output_path)
        
        step = step +1
        print("[Step %d] - Read files" % (step))

        for (_,_,files) in os.walk(asset_path):
            # count image
            count = 0
            print("Amount of images %d" % len(files))
            for file in files:

                # target the actual file path
                path_file = os.path.join(asset_path,file)

                # split file string, used when create new file
                file_splited = file.split(".")
                filename , mimetype = file_splited[0] , file_splited[1]
                if len(file_splited) == 4:
                    filename = file_splited[0] + file_splited[1] + file_splited[2]
                    mimetype = file_splited[3]
                
                
                # you can access to pixel of image here
                image = cv2.imread(path_file)
                # print(image)
                # print(np.array(image).shape)
                step = step +1
                print("[Step %d] - Rotating Image ...%s" % (step,filename))
                for angle in angles:
                    new_filename = "%s_rotated_%d.%s" % (filename,angle,mimetype)
                    new_filepath = os.path.join(output_path,new_filename)
                    # check if file was rotated, so you dont need to do it again.
                    if os.path.isfile(new_filepath):
                        print("%s was skipped !" % new_filename)
                        continue
                    
                    # rotated
                    count = count +1
                    print("Rotated %s at angle %d" % (filename, angle))
                    if angle == 0:
                        cv2.imwrite(new_filepath,image)
                        continue
                    
                   
                    image_rotated =  imutils.rotate(image,angle = angle)
                    image_rotated
                    cv2.imwrite(new_filepath,image_rotated)
                    
                
                print("After Rotated we get new amount of images: %d" % count)

In [6]:
def work_enhancement(input_array):
    for folder in input_array:
        step = 1
        print("[Info] Access to %s ..." % (folder["name"]))
        asset_path = folder['input']
        output_path = os.path.join("output","mango","enhancement",folder["name"])
        
        # check if output folder is existing, so we can create new one!
        print("[Step %d] - Checking output folder..." % (step))
        if not os.path.isdir(output_path):
            print("[Error Step %d] - Output path is not exist, Please running code after folder creation are done." % (step))
            os.makedirs(output_path)
        
        step = step +1
        print("[Step %d] - Read files" % (step))

        for (_,_,files) in os.walk(asset_path):
            for file in files:

                # target the actual file path
                path_file = os.path.join(asset_path,file)

                # split file string, used when create new file
                file_splited = file.split(".")
                filename , mimetype = file_splited[0] , file_splited[1]
                if len(file_splited) == 4:
                    filename = file_splited[0] + file_splited[1] + file_splited[2]
                    mimetype = file_splited[3]
                
                # you can access to pixel of image here
                image = cv2.imread(path_file)
                # print(image)
                # print(np.array(image).shape)
                step = step +1
                print("[Step %d] - Enhancing Image ...%s" % (step,filename))
                new_filename = "%s_clahe_enhanced.%s" % (filename,mimetype)
                new_filepath = os.path.join(output_path,new_filename)

                if os.path.isfile(new_filepath):
                    print("%s was skipped !" % new_filename)
                    continue

                enhanced = clahe(image)
                cv2.imwrite(new_filepath,enhanced)

In [7]:
def work_augmentation_new(input_array):
    # angles = get_angles(5)
    angles = get_angles(4)
    for folder in input_array:
        step = 1
        print("[Info] Access to %s ..." % (folder["name"]))
        asset_path = folder['input']
        output_path = os.path.join("output","mango","augmentation-enhancement",folder["name"])
        
        # check if output folder is existing, so we can create new one!
        print("[Step %d] - Checking output folder..." % (step))
        if not os.path.isdir(output_path):
            print("[Error Step %d] - Output path is not exist, Please running code after folder creation are done." % (step))
            os.makedirs(output_path)
        
        step = step +1
        print("[Step %d] - Read files" % (step))

        for (_,_,files) in os.walk(asset_path):
            # count image
            count = 0
            print("Amount of images %d" % len(files))
            for file in files:

                # target the actual file path
                path_file = os.path.join(asset_path,file)

                # split file string, used when create new file
                file_splited = file.split(".")
                filename , mimetype = file_splited[0] , file_splited[1]
                if len(file_splited) == 4:
                    filename = file_splited[0] + file_splited[1] + file_splited[2]
                    mimetype = file_splited[3]
                
                
                # you can access to pixel of image here
                image = cv2.imread(path_file)
                # print(image)
                # print(np.array(image).shape)
                step = step +1
                print("[Step %d] - Rotating Image ...%s" % (step,filename))
                for angle in angles:
                    new_filename = "%s_rotated_%d.%s" % (filename,angle,mimetype)
                    new_filepath = os.path.join(output_path,new_filename)
                    # check if file was rotated, so you dont need to do it again.
                    if os.path.isfile(new_filepath):
                        print("%s was skipped !" % new_filename)
                        continue
                    
                    # rotated
                    count = count +1
                    print("Rotated %s at angle %d" % (filename, angle))
                    if angle == 0:
                        cv2.imwrite(new_filepath,image)
                        continue
                    
                   
                    image_rotated =  imutils.rotate_bound(image,angle = angle)
                    image_rotated
                    cv2.imwrite(new_filepath,image_rotated)
                    
                
                print("After Rotated we get new amount of images: %d" % count)

In [8]:
from scipy.ndimage import rotate as rotate_image
def work_augmentation_new_ndimage(input_array):
    # angles = get_angles(5)
    angles = get_angles(3)
    for folder in input_array:
        step = 1
        print("[Info] Access to %s ..." % (folder["name"]))
        asset_path = folder['input']
        output_path = os.path.join("output","mango","augmentation-enhancement",folder["name"])
        
        # check if output folder is existing, so we can create new one!
        print("[Step %d] - Checking output folder..." % (step))
        if not os.path.isdir(output_path):
            print("[Error Step %d] - Output path is not exist, Please running code after folder creation are done." % (step))
            os.makedirs(output_path)
        
        step = step +1
        print("[Step %d] - Read files" % (step))

        for (_,_,files) in os.walk(asset_path):
            # count image
            count = 0
            print("Amount of images %d" % len(files))
            for file in files:

                # target the actual file path
                path_file = os.path.join(asset_path,file)

                # split file string, used when create new file
                file_splited = file.split(".")
                filename , mimetype = file_splited[0] , file_splited[1]
                if len(file_splited) == 4:
                    filename = file_splited[0] + file_splited[1] + file_splited[2]
                    mimetype = file_splited[3]
                
                
                # you can access to pixel of image here
                image = cv2.imread(path_file)
                # print(image)
                # print(np.array(image).shape)
                step = step +1
                print("[Step %d] - Rotating Image ...%s" % (step,filename))
                for angle in angles:
                    new_filename = "%s_rotated_%d.%s" % (filename,angle,mimetype)
                    new_filepath = os.path.join(output_path,new_filename)
                    # check if file was rotated, so you dont need to do it again.
                    if os.path.isfile(new_filepath):
                        print("%s was skipped !" % new_filename)
                        continue
                    
                    # rotated
                    # square_image = make_square(image, max(image.shape[:2]))
                    count = count +1
                    print("Rotated %s at angle %d" % (filename, angle))
                    if angle == 0:
                        cv2.imwrite(new_filepath,image)
                        continue
                    
                   
                     
                    image_rotated =  rotate_image(image,angle = angle)
                    image_rotated
                    cv2.imwrite(new_filepath,image_rotated)
                    
                
                print("After Rotated we get new amount of images: %d" % count)

In [9]:
from scipy.ndimage import rotate as rotate_image
def work_square_image(input_array):
    # angles = get_angles(5)
    angles = get_angles(4)
    for folder in input_array:
        step = 1
        print("[Info] Access to %s ..." % (folder["name"]))
        asset_path = folder['input']
        output_path = os.path.join("output","mango","augmentation-enhancement",folder["name"])
        
        # check if output folder is existing, so we can create new one!
        print("[Step %d] - Checking output folder..." % (step))
        if not os.path.isdir(output_path):
            print("[Error Step %d] - Output path is not exist, Please running code after folder creation are done." % (step))
            os.makedirs(output_path)
        
        step = step +1
        print("[Step %d] - Read files" % (step))

        for (_,_,files) in os.walk(asset_path):
            # count image
            count = 0
            print("Amount of images %d" % len(files))
            for file in files:

                # target the actual file path
                path_file = os.path.join(asset_path,file)

                # split file string, used when create new file
                file_splited = file.split(".")
                filename , mimetype = file_splited[0] , file_splited[1]
                if len(file_splited) == 4:
                    filename = file_splited[0] + file_splited[1] + file_splited[2]
                    mimetype = file_splited[3]
                
                
                # you can access to pixel of image here
                image = cv2.imread(path_file)
                # print(image)
                # print(np.array(image).shape)
                step = step +1
                square_image = make_square(image, max(image.shape[:2]))
                new_filename = "%s_rotated_%d.%s" % (filename,0,mimetype)
                new_filepath = os.path.join(output_path,new_filename)
                cv2.imwrite(new_filepath,square_image)
                # print("[Step %d] - Rotating Image ...%s" % (step,filename))
                # for angle in angles:
                #     new_filename = "%s_rotated_%d.%s" % (filename,angle,mimetype)
                #     new_filepath = os.path.join(output_path,new_filename)
                #     # check if file was rotated, so you dont need to do it again.
                #     if os.path.isfile(new_filepath):
                #         print("%s was skipped !" % new_filename)
                #         continue
                    
                #     # rotated
                #     square_image = make_square(image, max(image.shape[:2]))
                #     count = count +1
                #     print("Rotated %s at angle %d" % (filename, angle))
                #     if angle == 0:
                #         cv2.imwrite(new_filepath,square_image)
                #         continue
                    
                   
                     
                #     image_rotated =  rotate_image(square_image,angle = angle)
                #     image_rotated
                #     cv2.imwrite(new_filepath,square_image)
                    
                
                print("After Rotated we get new amount of images: %d" % count)

In [10]:
input_folder = [
            {
        'name' : "M1MHN",
        "input" : os.path.join("Test01","M1MHN"),
#         "input" : os.path.join("images","MHN")
    },
            {
        'name' : "M1NDM",
        "input" : os.path.join("Test01","M1NDM"),
#         "input" : os.path.join("images","NDM")
    },
            {
        'name' : "M1R2E2",
        "input" : os.path.join("Test01","M1R2E2"),
#         "input" : os.path.join("images","R2E2")
    },
            {
        'name' : "M2MHN",
        "input" : os.path.join("Test01","M2MHN"),
#         "input" : os.path.join("images","R2E2")
    },
            {
        'name' : "M2NDM",
        "input" : os.path.join("Test01","M2NDM"),
#         "input" : os.path.join("images","R2E2")
    },
            {
        'name' : "M2R2E2",
        "input" : os.path.join("Test01","M2R2E2"),
#         "input" : os.path.join("images","R2E2")
    },
            {
        'name' : "M3MHN",
        "input" : os.path.join("Test01","M3MHN"),
#         "input" : os.path.join("images","MHN")
    },
            {
        'name' : "M3NDM",
        "input" : os.path.join("Test01","M3NDM"),
#         "input" : os.path.join("images","NDM")
    },

]

# work_augmentation(input_folder)
# work_augmentation_new(input_folder)
# work_augmentation_new_ndimage(input_folder)
work_square_image(input_folder)
# work_enhancement(input_folder)

[Info] Access to M1MHN ...
[Step 1] - Checking output folder...
[Error Step 1] - Output path is not exist, Please running code after folder creation are done.
[Step 2] - Read files
Amount of images 98
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of images: 0
After Rotated we get new amount of